<a href="https://colab.research.google.com/github/perlatomdpi/GPU-Machine-Learning-Algorithms/blob/main/GPU_Geographics_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GeoGraphics netwotk analysis**
From geographic data we can obrain distance between the cities (nodes) that ae connected (edges), and the size of the edges could be the averaged limit-speed to arrive from source node (departure city) to destination node (destitation city).

# **Libraries**

In [ ]:
# After installing RAPIDs impoorrt the following acceletated libraries + networkx
import cudf
import cugraph as cg
import networkx as nx

In [ ]:
# Read data
road_nodes = cudf.read_csv('./data/node.csv')
road_edges = cudf.read_csv('./data/edge.csv')

In [ ]:
# summary the size of the network
print(f'{road_edges.shape[0]} edges, {road_nodes.shape[0]} nodes')

# **Create the network**

In [ ]:
G = cg.Graph()
%time G.from_cudf_edgelist(road_edges, source='src_id', destination='dst_id', edge_attr='length')

# Reindex nodes
# For efficient lookup, we will reindex road_nodes to use the node_id as its index
road_nodes = road_nodes.set_index('node_id', drop=True)
%time road_nodes = road_nodes.sort_index()
road_nodes.head()

# Calculate degree centrality
deg_df = G.degree()
deg_df['degree'].describe()[1:]

# **Speed-limit used as Weight into the Geographic Network**

In [ ]:
speed_limits = {'Motorway': 70,
               'A Road': 60,
               'B Road': 60,
               'Local Road': 30,
               'Local Access Road': 30,
               'Minor Road': 30,
               'Restricted Local Access Road': 30,
               'Secondary Access Road': 30}

In [ ]:
# Create cuDf and store the speed limit per each road type
speed_gdf = cudf.DataFrame()

speed_gdf['type'] = speed_limits.keys() # Dictionaryy key of the df speed_limits
speed_gdf['limit_mph'] = [speed_limits[key] for key in speed_limits.keys()]
speed_gdf

In [ ]:
# Add an additional column called "limit_m/s" which for each road type will give us a measure of how fast one can travel on it in meters / second
# Road distances in meters (m) --> road distances in seconds (s), 
# We need to multiply by meters/mile and divide by seconds/hour
# 1 mile ~ 1609.34 m
speed_gdf['limit_m/s'] = speed_gdf['limit_mph'] * 1609.34 / 3600
speed_gdf

# **Merge speed_gdf into road_edges**

In [ ]:
# cuDF provides merging functionality just like Pandas
%time road_edges = road_edges.merge(speed_gdf, on='type')

# **Add Length in Seconds Column**

In [ ]:
# Calcualte the distance between connected cities
road_edges['length_s'] = road_edges['length'] / road_edges['limit_m/s']
road_edges['length_s'].head()

In [ ]:
# Create cuGraph and use speed-limit as weight
G_ex = cg.Graph()
G_ex.from_cudf_edgelist(road_edges, source='src_id', destination='dst_id', edge_attr='limit_m/s')